### Hemati DMD

- vecina koda prilagodeno od https://github.com/cwrowley/dmdtools

In [1]:
import numpy as np

In [ ]:
def Hemati_update(Qx, Qy, A, Gx, Gy, x, y, ngram=5, epsilon=1e-10, r0=0):

    #reortogonalizacija
    rx=Qx.shape[1]; ry=Qy.shape[1]; n=x.shape[0]
    xtilde = np.matrix(np.zeros((rx, 1)))
    ytilde = np.matrix(np.zeros((ry, 1)))
    ex = np.matrix(x).reshape((n, 1))
    ey = np.matrix(y).reshape((n, 1))
    for _ in range(ngram):
        dx = Qx.T@ex
        dy = Qy.T@ey
        xtilde += dx
        ytilde += dy
        ex -= Qx.dot(dx)
        ey -= Qy.dot(dy)
    
    #prosirivanje - ako potrebno
        #za x
    norm_ex=np.linalg.norm(ex); norm_ey = np.linalg.norm(ey)
    normx=np.linalg.norm(x); normy = np.linalg.norm(y)
    if norm_ex / normx > epsilon:
        Qx = np.bmat([Qx, ex/np.linalg.norm(ex)])
        Gx = np.bmat([[Gx, np.zeros((rx, 1))], [np.zeros((1, rx+1))]])
        A = np.bmat([A, np.zeros((ry, 1))])
        rx += 1


        #za y
    if norm_ey / normy > epsilon:
        Qy = np.bmat([Qy, ey/np.linalg.norm(ey)])
        Gy = np.bmat([[Gy, np.zeros((ry, 1))], [np.zeros((1, ry+1))]])
        A = np.bmat([[A],[np.zeros((1, rx))]])
        ry += 1
    
    #pod compression - dodano - 
    if r0:
        if rx > r0:
            evals, evecs = np.linalg.eig(Gx)
            idx = np.argsort(evals)
            idx = idx[-1:-1-r0:-1]   # indices of largest r0 eigenvalues
            qx = np.asmatrix(evecs[:, idx])
            Qx = Qx * qx
            A = A * qx
            Gx = np.asmatrix(np.diag(evals[idx]))
        if ry > r0:
            evals, evecs = np.linalg.eig(Gy)
            idx = np.argsort(evals)
            idx = idx[-1:-1-r0:-1]   # indices of largest r0 eigenvalues
            qy = np.asmatrix(evecs[:, idx])
            Qy = Qy * qy
            A = qy.T * A
            Gy = np.asmatrix(np.diag(evals[idx]))


    #update A and Gx
    xtilde = Qx.T * x
    ytilde = Qy.T * y

    # update A and Gx
    A += ytilde * xtilde.T
    Gx += xtilde * xtilde.T
    Gy += ytilde * ytilde.T

    return Qx, Qy, A, Gx, Gy


In [ ]:
def Hemati_compute_modes(Qx, Qy, A, Gx):
    """takes Qx, Qy, A, Gx from Hemati et al. paper. 
    Returns Z (koopman eigenmodes) and Lambda (koopman eigenvalues)"""
    S_k = Qx.T@Qy@A@np.linalg.pinv(Gx)
    Lambda, W = np.linalg.eig(S_k)
    Z = Qx @ W 
    #r = ...
    return Z, Lambda